In [2]:
import pandas as pd
import plotly.express as px
import os 
import pathlib

print(os.getcwd())
print(pathlib.Path.cwd())

data = pd.read_excel('data/get_around_delay_analysis.xlsx')
# for index, i in enumerate(data.columns):
#     print(f"{data.dtypes.iloc[index]} --> {i} ")
print(data.dtypes)
print()
display(data.head())
display(data.shape)

c:\Users\bonna\Desktop\CREA_APPLICATIONS\dsfs-od-11\5 - deployment\eda_getaround
c:\Users\bonna\Desktop\CREA_APPLICATIONS\dsfs-od-11\5 - deployment\eda_getaround
rental_id                                       int64
car_id                                          int64
checkin_type                                   object
state                                          object
delay_at_checkout_in_minutes                  float64
previous_ended_rental_id                      float64
time_delta_with_previous_rental_in_minutes    float64
dtype: object



,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
0,505000,363965,mobile,canceled,NaN,NaN,NaN
1,507750,269550,mobile,ended,-81.0,NaN,NaN
2,508131,359049,connect,ended,70.0,NaN,NaN
3,508865,299063,connect,canceled,NaN,NaN,NaN
4,511440,313932,mobile,ended,NaN,NaN,NaN


(21310, 7)

In [3]:
# Pour comprendre les données, voir une seule voiture
data[data['car_id'] == 359049].sort_values('rental_id')

,rental_id,car_id,checkin_type,state,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
2,508131,359049,connect,ended,70.0,NaN,NaN
9754,537126,359049,connect,ended,-156.0,539408.0,720.0
13600,539408,359049,connect,ended,-6.0,NaN,NaN
8219,544433,359049,connect,ended,-110.0,NaN,NaN
6391,546894,359049,connect,ended,2.0,NaN,NaN
15866,547579,359049,connect,ended,32.0,NaN,NaN
17483,550645,359049,connect,ended,249.0,NaN,NaN
8992,553735,359049,connect,ended,33.0,550645.0,390.0
5697,561550,359049,connect,canceled,NaN,NaN,NaN
14550,563861,359049,connect,canceled,NaN,NaN,NaN


In [4]:
display(data.isna().sum())
display(data["state"].value_counts())


rental_id                                         0
car_id                                            0
checkin_type                                      0
state                                             0
delay_at_checkout_in_minutes                   4964
previous_ended_rental_id                      19469
time_delta_with_previous_rental_in_minutes    19469
dtype: int64

state
ended       18045
canceled     3265
Name: count, dtype: int64

In [5]:
display(data.describe().round(2))

,rental_id,car_id,delay_at_checkout_in_minutes,previous_ended_rental_id,time_delta_with_previous_rental_in_minutes
count,21310.00,21310.00,16346.00,1841.00,1841.00
mean,549712.88,350030.60,59.70,550127.41,279.29
std,13863.45,58206.25,1002.56,13184.02,254.59
min,504806.00,159250.00,-22433.00,505628.00,0.00
25%,540613.25,317639.00,-36.00,540896.00,60.00
50%,550350.00,368717.00,9.00,550567.00,180.00
75%,560468.50,394928.00,67.00,560823.00,540.00
max,576401.00,417675.00,71084.00,575053.00,720.00


# Description des données : 

## Features 

- ###  delay_at_checkout_in_minutes

Temps réel du checkin de la reservation : 

- Si supérieur à zéro => Checkin en retard
- Si inférieur à zéro => Checkin en avance

- ###  time_delta_with_previous_rental_in_minutes 

Temps estimé entre les deux reservations, c'est à dire le delta entre l'horaire de fin programmé et l'horaire de début programmé de la reservation.

Si NaN, pas d'enchainement de reservation donc pas de problème de retard



In [6]:
print("Distribution des ended et canceled :")
print()
print(f"% de 'ended' : {round((data["state"].value_counts().iloc[0] / data.shape[0])*100,2)} %")
print(f"% de 'canceled' : {round((data["state"].value_counts().iloc[1] / data.shape[0])*100,2)} %")
plot = px.bar(data.value_counts(subset='state').reset_index(),x='state',y='count',title="Distribution ended / canceled",width=400)
plot.show()

Distribution des ended et canceled :

% de 'ended' : 84.68 %
% de 'canceled' : 15.32 %


In [7]:
print(f"% de 'mobile' : {round((data["checkin_type"].value_counts().iloc[0] / data.shape[0])*100,2)} %")
print(f"% de 'connect' : {round((data["checkin_type"].value_counts().iloc[1] / data.shape[0])*100,2)} %")
plot = px.bar(data.value_counts(subset='checkin_type').reset_index(),x='checkin_type',y='count',title="Distribution mobile / connect",width=400)
plot.show()

% de 'mobile' : 79.79 %
% de 'connect' : 20.21 %


In [8]:
print("Distribution des ended et canceled quand enchainement :")
print("")
print(f"% de 'ended' : {round((data[data['previous_ended_rental_id'].notna()]["state"].value_counts().iloc[0] / data.shape[0])*100,2)} %")
print(f"% de 'canceled' : {round((data[data['previous_ended_rental_id'].notna()]["state"].value_counts().iloc[1] / data.shape[0])*100,2)} %")
plot = px.bar(data[data['previous_ended_rental_id'].notna()].value_counts(subset='state').reset_index(),x='state',y='count',title="Distribution ended / canceled (enchainement des reservations)",width=400)
plot.show()
print("")
print("Distribution des ended et canceled quand enchainement :")
print("")
print(f"% de 'mobile' : {round((data[data['previous_ended_rental_id'].notna()]["checkin_type"].value_counts().iloc[0] / data.shape[0])*100,2)} %")
print(f"% de 'canconnectceled' : {round((data[data['previous_ended_rental_id'].notna()]["checkin_type"].value_counts().iloc[1] / data.shape[0])*100,2)} %")
plot1 = px.bar(data[data['previous_ended_rental_id'].notna()].value_counts(subset='checkin_type').reset_index(),x='checkin_type',y='count',title="Distribution mobile / connect (enchainement des reservations)",width=400)
plot1.show()

Distribution des ended et canceled quand enchainement :

% de 'ended' : 7.56 %
% de 'canceled' : 1.07 %



Distribution des ended et canceled quand enchainement :

% de 'mobile' : 4.82 %
% de 'canconnectceled' : 3.82 %


In [9]:
data_ended = data[data['state'] == "ended"].copy()
print(f"Shape des 'ended' : {data_ended.shape}")
# on enlève les NaN car sinon pas d'enchainement de reservations
multiple_reservations = data_ended[(data_ended['previous_ended_rental_id'].notna()) & (data_ended['delay_at_checkout_in_minutes'].notna())].copy()
multiple_reservations['delay_at_checkout_in_minutes'] = multiple_reservations['delay_at_checkout_in_minutes'].fillna(value=0)
multiple_reservations['se_chevauche'] = multiple_reservations['delay_at_checkout_in_minutes'] - multiple_reservations['time_delta_with_previous_rental_in_minutes']
multiple_reservations['en_retard'] = (multiple_reservations['se_chevauche'] > 0)

print(f"Nouvelle shape en ne gardant que les non NaN : {multiple_reservations.shape}")
print("")
print(f"Nombre de réservations que les retards affectent : {multiple_reservations[multiple_reservations['en_retard']]['rental_id'].count()} retards, soit : {(multiple_reservations[multiple_reservations['en_retard']]['rental_id'].count())/(data.shape[0])*100:.2f} % du total des reservations")
print(f"Nombre de réservations que les retards affectent : {multiple_reservations[multiple_reservations['en_retard']]['rental_id'].count()} retards, soit : {(multiple_reservations[multiple_reservations['en_retard']]['rental_id'].count())/(data_ended.shape[0])*100:.2f} % du total des reservations 'ended'")
print(f"Nombre de réservations que les retards affectent : {multiple_reservations[multiple_reservations['en_retard']]['rental_id'].count()} retards, soit : {(multiple_reservations[multiple_reservations['en_retard']]['rental_id'].count())/(multiple_reservations.shape[0])*100:.2f} % du total des reservations qui s'enchainent")

print("")
print("Description de la colonne 'se_chevauche' (c'est à dire que le délai entre deux reservations est insuffisants par rapport au temps de checkin) :")
desc_drivers_late = multiple_reservations[multiple_reservations['en_retard']][['se_chevauche']].describe()
print(f"{desc_drivers_late}")
print(f"Moyenne des retards : {multiple_reservations['delay_at_checkout_in_minutes'].mean():.2f} minutes en retard")
# print(f"Les conducteurs sont en moyenne : {multiple_reservations[multiple_reservations['checkin_type'] == 'connect']['delay_at_checkout_in_minutes'].mean():.2f} minutes en retard à la prochaine réservation")
# print(f"Les conducteurs sont en moyenne : {multiple_reservations[multiple_reservations['checkin_type'] == 'mobile']['delay_at_checkout_in_minutes'].mean():.2f} minutes en retard à la prochaine réservation")

Shape des 'ended' : (18045, 7)
Nouvelle shape en ne gardant que les non NaN : (1515, 9)

Nombre de réservations que les retards affectent : 270 retards, soit : 1.27 % du total des reservations
Nombre de réservations que les retards affectent : 270 retards, soit : 1.50 % du total des reservations 'ended'
Nombre de réservations que les retards affectent : 270 retards, soit : 17.82 % du total des reservations qui s'enchainent

Description de la colonne 'se_chevauche' (c'est à dire que le délai entre deux reservations est insuffisants par rapport au temps de checkin) :
       se_chevauche
count    270.000000
mean     289.203704
std      815.822763
min        1.000000
25%       23.250000
50%       64.500000
75%      186.750000
max     9697.000000
Moyenne des retards : 27.53 minutes en retard


In [10]:
thresolds = [15,25,30,45,60,75,120]
results_all = []
for thresold in thresolds : 
    problems = multiple_reservations[multiple_reservations['en_retard']]
    ## 119 € provient de le médiane des prix 
    print(f"Nombre de réservations impactées par le problème : {problems.shape[0]} pour {problems.shape[0]*119} €")
    solved_by_thresold = problems[problems['time_delta_with_previous_rental_in_minutes'] < thresold]
    not_solved_by_thresold =  problems.shape[0] - solved_by_thresold.shape[0]
    print(f"Le thresold de {thresold} minutes masquerait : {solved_by_thresold.shape[0]} des véhicules soit : {(solved_by_thresold.shape[0] / problems.shape[0])*100:.2f} % des reservations à problème")
    print(f"cela représente : {(solved_by_thresold.shape[0] / data_ended.shape[0])*100:.2f} % des ended au total")
    print(f"et : {(solved_by_thresold.shape[0] / multiple_reservations.shape[0])*100:.2f} % des ended avec plusieurs reservations")
    print("")



Nombre de réservations impactées par le problème : 270 pour 32130 €
Le thresold de 15 minutes masquerait : 136 des véhicules soit : 50.37 % des reservations à problème
cela représente : 0.75 % des ended au total
et : 8.98 % des ended avec plusieurs reservations

Nombre de réservations impactées par le problème : 270 pour 32130 €
Le thresold de 25 minutes masquerait : 136 des véhicules soit : 50.37 % des reservations à problème
cela représente : 0.75 % des ended au total
et : 8.98 % des ended avec plusieurs reservations

Nombre de réservations impactées par le problème : 270 pour 32130 €
Le thresold de 30 minutes masquerait : 136 des véhicules soit : 50.37 % des reservations à problème
cela représente : 0.75 % des ended au total
et : 8.98 % des ended avec plusieurs reservations

Nombre de réservations impactées par le problème : 270 pour 32130 €
Le thresold de 45 minutes masquerait : 176 des véhicules soit : 65.19 % des reservations à problème
cela représente : 0.98 % des ended au total

In [11]:
thresolds = [15,25,30,45,60,75,120]
results_connect = []
problems = multiple_reservations[multiple_reservations['en_retard']]
problems_connect = problems[problems['checkin_type'] == "connect"]
print(f"Nombre de réservations impactées par le problème sur les voitures 'connect' : {problems_connect.shape[0]} sur {problems.shape[0]} : {(problems_connect.shape[0]/problems.shape[0])*100:.2f} %")
print("")
for thresold in thresolds : 
    solved_by_thresold = problems_connect[problems_connect['time_delta_with_previous_rental_in_minutes'] < thresold]
    not_solved_by_thresold =  problems_connect.shape[0] - solved_by_thresold.shape[0]
    print(f"Le thresold de {thresold} minutes masquerait : {solved_by_thresold.shape[0]} des véhicules en connect soit : {(solved_by_thresold.shape[0] / problems.shape[0])*100:.2f} % des reservations à problème")
    print(f"cela représente : {(solved_by_thresold.shape[0] / data_ended.shape[0])*100:.2f} % des ended au total")
    print(f"et : {(solved_by_thresold.shape[0] / multiple_reservations.shape[0])*100:.2f} % des ended avec plusieurs reservations")
    print("")



Nombre de réservations impactées par le problème sur les voitures 'connect' : 80 sur 270 : 29.63 %

Le thresold de 15 minutes masquerait : 52 des véhicules en connect soit : 19.26 % des reservations à problème
cela représente : 0.29 % des ended au total
et : 3.43 % des ended avec plusieurs reservations

Le thresold de 25 minutes masquerait : 52 des véhicules en connect soit : 19.26 % des reservations à problème
cela représente : 0.29 % des ended au total
et : 3.43 % des ended avec plusieurs reservations

Le thresold de 30 minutes masquerait : 52 des véhicules en connect soit : 19.26 % des reservations à problème
cela représente : 0.29 % des ended au total
et : 3.43 % des ended avec plusieurs reservations

Le thresold de 45 minutes masquerait : 63 des véhicules en connect soit : 23.33 % des reservations à problème
cela représente : 0.35 % des ended au total
et : 4.16 % des ended avec plusieurs reservations

Le thresold de 60 minutes masquerait : 63 des véhicules en connect soit : 23.33 

In [12]:
print("Taux de retard en fonction du device :")
mobile = data_ended[data_ended['checkin_type'] == 'mobile'].copy()
connect = data_ended[data_ended['checkin_type'] == 'connect'].copy()
print(f"Shape de mobile : {mobile.shape}")
print(f"Shape de connect : {connect.shape}")
print("")
print("On ne garde que les non NaN :")

mobile = mobile[mobile['time_delta_with_previous_rental_in_minutes'].notna()]
connect = connect[connect['time_delta_with_previous_rental_in_minutes'].notna()]
print(f"Nouvelle shape de mobile : {mobile.shape}")
print(f"Nouvelle shape de connect : {connect.shape}")
print("")

mobile['delay_at_checkout_in_minutes'] = mobile['delay_at_checkout_in_minutes'].fillna(value=0)
connect['delay_at_checkout_in_minutes'] = connect['delay_at_checkout_in_minutes'].fillna(value=0)

mobile['conflit'] = mobile['delay_at_checkout_in_minutes'] - mobile['time_delta_with_previous_rental_in_minutes']
connect['conflit'] = connect['delay_at_checkout_in_minutes'] - connect['time_delta_with_previous_rental_in_minutes']

mobile['en_retard'] = (mobile['conflit'] > 0)
connect['en_retard'] = (connect['conflit'] > 0)

print(f"{(mobile[mobile['en_retard']]['car_id'].count()) / (mobile.shape[0]) * 100:.2f} % des reservations 'mobile' qui s'enchainent sont en retard/impactées")
print(f"{(connect[connect['en_retard']]['car_id'].count()) / (connect.shape[0]) * 100:.2f} % des reservations 'connect' qui s'enchainent sont en retard/impactées")

print(f"{(mobile[mobile['en_retard']]['car_id'].count()) / (data_ended.shape[0]) * 100:.2f} % des reservations 'mobile' qui s'enchainent sont en retard/impactées sur le total")
print(f"{(connect[connect['en_retard']]['car_id'].count()) / (data_ended.shape[0]) * 100:.2f} % des reservations 'connect' qui s'enchainent sont en retard/impactées sur le total")

Taux de retard en fonction du device :
Shape de mobile : (14536, 7)
Shape de connect : (3509, 7)

On ne garde que les non NaN :
Nouvelle shape de mobile : (930, 7)
Nouvelle shape de connect : (682, 7)

20.43 % des reservations 'mobile' qui s'enchainent sont en retard/impactées
11.73 % des reservations 'connect' qui s'enchainent sont en retard/impactées
1.05 % des reservations 'mobile' qui s'enchainent sont en retard/impactées sur le total
0.44 % des reservations 'connect' qui s'enchainent sont en retard/impactées sur le total


Quel est la part des reservation qui n'aurait pas été possible si Threshold = 15 minutes, 30 minutes, 1 heure, etc

Sachant que la moyenne des retard est de : 

In [13]:
# How often are drivers late for the next check-in? How does it impact the next driver?

print(f"Nombre de conducteur en retard au chekin précedent : {data[data["time_delta_with_previous_rental_in_minutes"] > 0].count()['rental_id']}")
print(f"Nombre de conducteur en retard au chekin précedent et en retard au checkin : {data[(data["time_delta_with_previous_rental_in_minutes"] > 0) & (data['delay_at_checkout_in_minutes'].notna() > 0)].count()['rental_id']}")
print(f"Les reservations qui sont en retard au checkin  : {data[(data["time_delta_with_previous_rental_in_minutes"] > 0) & (data['delay_at_checkout_in_minutes'].notna() > 0)]['delay_at_checkout_in_minutes'].mean().round(2)}")

Nombre de conducteur en retard au chekin précedent : 1562
Nombre de conducteur en retard au chekin précedent et en retard au checkin : 1288
Les reservations qui sont en retard au checkin  : 24.23


Trouver les voitures qui ont beaucoup d'enchainement de reservation

In [14]:
top_car_reservation = data.groupby('car_id')['rental_id'].count().sort_values(ascending=False).reset_index().rename({'rental_id':'count'},axis=1)
print(f"{top_car_reservation}")
top10_nb_reservation = top_car_reservation[top_car_reservation['count'] > 10]
print(top10_nb_reservation.shape)

      car_id  count
0     334190     33
1     241021     29
2     392479     29
3     342699     27
4     300505     24
...      ...    ...
8138  416611      1
8139  416559      1
8140  415763      1
8141  415741      1
8142  415702      1

[8143 rows x 2 columns]
(166, 2)


In [15]:
reservation = data[data['car_id'] == 334190].sort_values('rental_id')
print(reservation[['rental_id','delay_at_checkout_in_minutes','time_delta_with_previous_rental_in_minutes']])

       rental_id  delay_at_checkout_in_minutes  \
2261      530615                           NaN   
927       532681                           NaN   
8886      534169                          25.0   
3823      536867                          26.0   
3174      537676                          39.0   
13438     538875                        -875.0   
1106      538906                           NaN   
2328      539414                        -286.0   
2706      541582                           NaN   
10398     542755                          60.0   
15804     543771                        -197.0   
15712     543805                        -130.0   
15235     543980                         -42.0   
15938     545023                           NaN   
8065      545590                           NaN   
10759     545600                        -922.0   
18577     546819                           NaN   
19249     546877                           NaN   
18586     547115                          93.0   


Intéressant de regarder le nombre d'enchainement de reservation entre les connects cars et le mobiles et aussi le time_delta entre mobile et connect

Pour voir si les connect card s'enchainent plus et si 